In [1]:
import os
from keras import applications
from keras import models
from keras import layers
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, Activation, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras import regularizers
import matplotlib.pyplot as plt
from keras.models import load_model
from keras.utils.np_utils import to_categorical 
from keras.callbacks import ModelCheckpoint
from keras.preprocessing import image
from keras import Model
from keras import initializers
from keras.callbacks import LearningRateScheduler
from keras.utils import layer_utils, np_utils
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.datasets import make_classification
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
from classification_models import ResNet18
from classification_models.resnet import preprocess_input as resnet_preprocess_input
from keras.applications.densenet import preprocess_input as densenet_preprocess_input
from keras.applications.vgg16 import preprocess_input as vgg_preprocess_input
from sklearn.cross_validation import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
from sklearn.utils import class_weight
import seaborn as sn
import pandas as pd
from scipy import interp
from itertools import cycle
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix
from sklearn.externals import joblib

Using TensorFlow backend.
c:\users\hp\appdata\local\programs\python\python36\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
image_width = 224
image_height = 224

# Change the batchsize according to your system RAM
batch_size = 64

train_dir = "D:/retinal_data_set_visioncare/New_Train_Test_Data/train/"
#test_dir = "D:/retinal_data_set_visioncare/New_Train_Test_Data/test/"

feature_scaler_filename = "D:/retinal_data_set_visioncare/Image_Retrieval/feature_scaler.save"
svd_scaler_file_name = "D:/retinal_data_set_visioncare/Image_Retrieval/svd_scaler.save"
file_path = 'D:/retinal_data_set_visioncare/Image_Retrieval/img_database.csv'

densenet_base = applications.DenseNet201(weights='imagenet', include_top=False, input_shape=(image_width, image_height, 3))
resnet_base = ResNet18(weights='imagenet', include_top=False, input_shape=(image_width, image_height, 3))
vgg_base = applications.VGG16(weights='imagenet', include_top=False, input_shape=(image_width, image_height, 3))

In [4]:
densenet_x = densenet_base.get_layer(index=-1).output
densenet_feature_extraction_layer = GlobalAveragePooling2D()(densenet_x)
densenet_model = Model(inputs=densenet_base.input, outputs=densenet_feature_extraction_layer)
densenet_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_21 (ZeroPadding2 (None, 230, 230, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d_21[0][0]          
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

__________________________________________________________________________________________________
conv5_block30_0_relu (Activatio (None, 7, 7, 1824)   0           conv5_block30_0_bn[0][0]         
__________________________________________________________________________________________________
conv5_block30_1_conv (Conv2D)   (None, 7, 7, 128)    233472      conv5_block30_0_relu[0][0]       
__________________________________________________________________________________________________
conv5_block30_1_bn (BatchNormal (None, 7, 7, 128)    512         conv5_block30_1_conv[0][0]       
__________________________________________________________________________________________________
conv5_block30_1_relu (Activatio (None, 7, 7, 128)    0           conv5_block30_1_bn[0][0]         
__________________________________________________________________________________________________
conv5_block30_2_conv (Conv2D)   (None, 7, 7, 32)     36864       conv5_block30_1_relu[0][0]       
__________

In [5]:
resnet_x = resnet_base.get_layer(index=-1).output
resnet_feature_extraction_layer = GlobalAveragePooling2D()(resnet_x)
resnet_model = Model(inputs=resnet_base.input, outputs=resnet_feature_extraction_layer)
resnet_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
bn_data (BatchNormalization)    (None, 224, 224, 3)  9           data[0][0]                       
__________________________________________________________________________________________________
zero_padding2d_23 (ZeroPadding2 (None, 230, 230, 3)  0           bn_data[0][0]                    
__________________________________________________________________________________________________
conv0 (Conv2D)                  (None, 112, 112, 64) 9408        zero_padding2d_23[0][0]          
__________________________________________________________________________________________________
bn0 (Batch

In [6]:
vgg_x = vgg_base.get_layer(index=-1).output
vgg_feature_extraction_layer = GlobalAveragePooling2D()(vgg_x)
vgg_model = Model(inputs=vgg_base.input, outputs=vgg_feature_extraction_layer)
vgg_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [7]:
class_labels = os.listdir(train_dir)

In [8]:
# Returns a compiled model identical to the previous one
loaded_pretrained_deep_feature_model = load_model('D:/retinal_data_set_visioncare/models/ensemble/densenet_deep_feature_with_SVD_dr.h5')
loaded_pretrained_deep_feature_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_25 (Dense)             (None, 128)               31872     
_________________________________________________________________
batch_normalization_25 (Batc (None, 128)               512       
_________________________________________________________________
activation_25 (Activation)   (None, 128)               0         
_________________________________________________________________
dropout_14 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_26 (Dense)             (None, 5)                 645       
_________________________________________________________________
batch_normalization_26 (Batc (None, 5)                 20        
_________________________________________________________________
activation_26 (Activation)   (None, 5)                 0         
Total para

In [9]:
feature_extraction_layer = loaded_pretrained_deep_feature_model.get_layer('activation_25').output
#feature_extraction_layer = GlobalAveragePooling2D()(x)
feature_extract_model = Model(inputs=loaded_pretrained_deep_feature_model.input, outputs=feature_extraction_layer)
feature_extract_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_25_input (InputLayer)  (None, 248)               0         
_________________________________________________________________
dense_25 (Dense)             (None, 128)               31872     
_________________________________________________________________
batch_normalization_25 (Batc (None, 128)               512       
_________________________________________________________________
activation_25 (Activation)   (None, 128)               0         
Total params: 32,384
Trainable params: 32,128
Non-trainable params: 256
_________________________________________________________________


In [10]:
# Returns a compiled model identical to the previous one
loaded_deep_hash_model = load_model('D:/retinal_data_set_visioncare/Image_Retrieval/deep_hash_model.h5')
loaded_deep_hash_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 28)                3612      
_________________________________________________________________
batch_normalization_5 (Batch (None, 28)                112       
_________________________________________________________________
activation_5 (Activation)    (None, 28)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 5)                 145       
_________________________________________________________________
batch_normalization_6 (Batch (None, 5)                 20        
_________________________________________________________________
activation_6 (Activation)    (None, 5)                 0         
Total params: 3,889
Trainable params: 3,823
Non-trainable params: 66
_________________________________________________________________


In [11]:
hashcode_extraction_layer = loaded_deep_hash_model.get_layer('activation_5').output
#feature_extraction_layer = GlobalAveragePooling2D()(x)
hashcode_extract_model = Model(inputs=loaded_deep_hash_model.input, outputs=hashcode_extraction_layer)
hashcode_extract_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7_input (InputLayer)   (None, 128)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 28)                3612      
_________________________________________________________________
batch_normalization_5 (Batch (None, 28)                112       
_________________________________________________________________
activation_5 (Activation)    (None, 28)                0         
Total params: 3,724
Trainable params: 3,668
Non-trainable params: 56
_________________________________________________________________


In [12]:
norm_scalar = joblib.load(feature_scaler_filename) 
norm_truncated_opt_svd = joblib.load(svd_scaler_file_name) 

In [43]:
def create_traditional_features(data_dir, file_path):
    COLUMN_NAMES=['img_path','deep_features','hash_code']
    df = pd.DataFrame(columns=COLUMN_NAMES)
    
    for idx, level in enumerate(class_labels):
        retina_root = data_dir + level + '/'
        retina_root_list = os.listdir(retina_root)

        for index, relative_path in enumerate(retina_root_list):
            if relative_path == 'Thumbs.db':
                continue
            source = retina_root + relative_path
            img = image.load_img(source, target_size=(image_width, image_height))
            img_x = image.img_to_array(img)
            img_x = np.expand_dims(img_x, axis=0)
            
            # densenet201 - feature extraction
            densenet201_x = densenet_preprocess_input(img_x)
            densenet201_extract_features = densenet_model.predict(densenet201_x)
            flattern_feature_vector = densenet201_extract_features.flatten()
            
            # resnet18 - feature extraction
            resnet18_x = resnet_preprocess_input(img_x)
            resnet18_extract_features = resnet_model.predict(resnet18_x)
            resnet18_feature_vector = resnet18_extract_features.flatten()
            
            # vgg16 - feature extraction
            vgg16_x = vgg_preprocess_input(img_x)
            vgg16_extract_features = vgg_model.predict(vgg16_x)
            vgg16_feature_vector = vgg16_extract_features.flatten()

            flattern_feature_vector = np.concatenate((flattern_feature_vector, resnet18_feature_vector, vgg16_feature_vector))
            scaled_flattern_feature_vector = norm_scalar.transform(np.array([flattern_feature_vector]))
            transformed_flattern_feature_vector = norm_truncated_opt_svd.transform(scaled_flattern_feature_vector)
            ensemble_compressed_feature = feature_extract_model.predict(transformed_flattern_feature_vector)
            ensemble_compressed_feature_np = np.array([ensemble_compressed_feature.flatten()])
            
            # load feature extractor model for sigmoid layer model as a feature extractor
            deep_hash_proba = hashcode_extract_model.predict(ensemble_compressed_feature_np)
            #print(deep_hash_proba.flatten())
            deep_hash_code = np.asarray([1 if val >= 0.5 else 0 for val in deep_hash_proba.flatten()], dtype=np.int64)
            df = df.append({'img_path': source, 'exact_deep_features': [ensemble_compressed_feature_np], 'hash_code': [deep_hash_code]}, ignore_index=True)
            
    df.to_csv(file_path, encoding='utf-8', index=False)

In [44]:
create_traditional_features(train_dir, file_path)